# QM9 Dataset
[reference examples](https://github.com/deepchem/deepchem/tree/master/examples/qm9)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import tensorflow as tf
import sklearn 
import deepchem as dc

In [ ]:
np.random.seed(123)
tf.set_random_seed(123)

In [ ]:
import pathlib
model_dir = pathlib.Path('../models/qm9')

**Multitask Network**  

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='Raw', split='stratified')
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
from dc.models import MultiTaskRegressor

def mtr_model_builder(model_params, model_dir):
    mtr_model = MultiTaskRegressor(model_dir, **model_params)
    return mtr_model

In [ ]:
metric = [
    dc.metrics.Metric(dc.metric.mean_absolute_error, mode="regression"),
    dc.metrics.Metric(dc.metric.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm9_tasks)
}

model = mtr_model_builder(params, model_dir)

print("training MTR model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MTR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores: [kcal/mol]")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores: [kcal/mol]")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores: [kcal/mol]")
print(test_scores)

**Graph Convolution**  

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='GraphConv', split='stratified')
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
from dc.models.tensorgraph.models.graph_models import GraphConvTensorGraph

def gc_model_builder(model_params, model_dir):
    return GraphConvTensorGraph(mode="regression", # tensorboard=True, 
                                model_dir=model_dir, 
                                **model_params)

In [ ]:
metric = [
    dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression"),
    dc.metrics.Metric(dc.metrics.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm9_tasks),
    "batch_size": 64,
    "learning_rate": 0.001
}

model = gc_model_builder(params, model_dir)

print("training GC model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating GC model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores: [kcal/mol]")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores: [kcal/mol]")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores: [kcal/mol]")
print(test_scores)

**Multitask Network** _(Coulomb Matrix Featurizer)_  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm9/qm9_tf_models.py)

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='stratified')
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
from dc.models import MultiTaskFitTransformRegressor

def mtftr_model_builder(model_params, model_dir):
    return MultitaskFitTransformRegressor(model_dir, **model_params)

In [ ]:
from deepchem.models.tensorgraph.optimizers import ExponentialDecay

fit_transformers = [
    dc.trans.CoulombFitTransformer(train_dataset)
]

metric = [
    dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression"),
    dc.metrics.Metric(dc.metrics.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm9_tasks),
    "n_features": [29, 29],
    "weight_init_stddevs": [1.0/np.sqrt(400), 1.0/np.sqrt(100), 1.0/np.sqrt(100)],
    "bias_init_consts": [0.0, 0.0, 0.0]
    "layer_sizes": [400, 100, 100],
    "dropouts": [0.01, 0.01, 0.01],
    "fit_transformers": fit_transformers,
    "batch_size": 32,
    "learning_rate": ExponentialDecay(0.001, 0.95, 1000),
    "momentum": 0.8,
    "n_evals": 10
}

model = mtftr_model_builder(params, model_dir)

print("training MTFTR model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MTFTR model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)

**Deep Tensor Neural Network**  
[reference example](https://github.com/deepchem/deepchem/blob/master/examples/qm9/qm9_DTNN.py)

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='CoulombMatrix', split='stratified')
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
def dtnn_model_builder(model_params, model_dir):
    pass # TODO

In [ ]:
metric = [
] # TODO

params = {
} # TODO

model = dtnn_model_builder(params, model_dir)

print("training DTNN model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating DTNN model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)

**Message Passing Neural Network**  

In [ ]:
qm9_tasks, qm9_datasets, transformers = dc.molnet.load_qm8_from_mat(
    featurizer='MP', split='stratified')
train_dataset, valid_dataset, test_dataset = qm9_datasets

In [ ]:
from dc.models import MPNNModel

def mpnn_model_builder(model_params, model_dir):
    return MPNNModel(use_queue=False,
                     mode="regression",
                     model_dir=model_dir,
                     **model_params)

In [ ]:
metric = [
    dc.metrics.Metric(dc.metrics.pearson_r2_score, mode="regression")
]

params = {
    "n_tasks": len(qm9_tasks),
    "n_atom_feat": 70,
    "n_pair_feat": 8,
    "T": 5,
    "M": 10,
    "batch_size": 32,
    "learning_reate": 0.0001
}

model = mpnn_model_builder(params, model_dir)

print("training MPNN model")
model.fit(train_dataset, nb_epochs=50)

print("evaluating MPNN model on {train,valid,test} sets")
train_scores = model.evaluate(train_dataset, metric, transformers)
print("train scores:")
print(train_scores)

valid_scores = model.evaluate(valid_dataset, metric, transformers)
print("valid scores:")
print(valid_scores)

test_scores = model.evaluate(test_dataset, metric, transformers)
print("test scores:")
print(test_scores)